In [108]:
# pip install geopy

In [109]:
# pip install operator

In [55]:
import pandas as pd
import numpy as np
import geopy
from geopy import distance
import operator

In [12]:
food_trucks_dataset=pd.read_excel('Downloads/Food_Trucks_dataset_Indianapolis_Xtern.xlsx')
df_food_trucks=pd.DataFrame(food_trucks_dataset)

In [13]:
df_food_trucks

,Sr no,Name,Address,Zipcode,Phone,Website,Cuisines,Latitude,Longitude,Ratings,Opening hour,Closing hour
0,1,Cluster Truck,"729 North Pennsylvania Street Indianapolis, IN",46204,(317) 732-1084,www.clustertruck.com,American,39.794471,-86.156289,4.8,8,22
1,2,General American Donut Company,"827 South East Street Indianapolis, IN",46225,(317) 379-7293,generalamericandonutco.com,Breakfast,39.755668,-86.149119,4.0,10,18
2,3,Johnsons BBQ Shack,"Indianapolis, IN",46204,(317) 716-8889,www.johnsonsbbqshack.com,BBQ,39.524465,-86.168535,4.5,10,18
3,4,Spice Box,222 East Market Street Booth 3 Indianapolis,46204,Unavailable,spiceboxindy.com,Indian,39.770235,-86.153376,4.3,10,20
4,5,Felix Tacos Truck,"3145 W Washington St, Indianapolis, IN",46222,(347)489-5554,Unavailable,Mexican,39.773102,-86.211039,4.5,17,24
5,6,La Marina Maxican Food Truck,"7155 Madison Ave, Indianapolis, IN",46227,Unavailable,Unavailable,Mexican,39.673286,-86.125208,4.0,16,22
6,7,Tortas Guicho Food Truck,"S. East St and, Fletcher Ave, Indianapolis, IN",46204,Unavailable,ordertortasguichodominguez.com,Mexican,39.773630,-86.152674,4.5,8,14
7,8,Pico de Gallo Food truck,"3014 Lafayette Rd #3000, Indianapolis, IN",46222,(317)658-6970,Unavailable,Mexican,39.819864,-86.217831,4.5,17,1
8,9,Big Jerry Indy Food truck,"8522 Madison Ave, Indianapolis, IN",46227,(317)331-9921,Unavailable,American,39.656372,-86.116282,5.0,11,19
9,10,Mi Lindo Penjamo,"7910 Michigan Rd, Indianapolis, IN",46268,(317)431-4439,store.kahero.co/milindopenjamo,Mexican,39.918624,-86.220147,4.5,19,24


## Since Cluster Truck opens at 8 am and is also rated at 4.8, we start our tour from there

In [17]:
Food_trucks=list(df_food_trucks['Name'])
latitudes=list(df_food_trucks['Latitude'])
longitudes=list(df_food_trucks['Longitude'])

In [42]:
def slope_calculator(lat_1,long_1,lat_2, long_2):
    #slope of a line , m = (y2-y1)/(x2-x1)
    #For small distances, the curvature of the earth can be ignored and latitude & longitude can be taken as coordinates on XY plane
    return (long_2-long_1)/(lat_2-lat_1)
    

In [45]:
slope_calculator(39.7944707585175, -86.1562887527601,39.7556679639579, -86.149118887406)

-0.18477703566138062

In [39]:
type((39.7556679639579, -86.149118887406))

tuple

In [104]:
def Identify_top_5_closest (first_stop_name, df_food_trucks):
    distances_from_first_stop=[]
    first_stop= (df_food_trucks['Latitude'][df_food_trucks['Name']==first_stop_name].iloc[0],df_food_trucks['Longitude'][df_food_trucks['Name']==first_stop_name].iloc[0] )
    
    for i in range (0, len(df_food_trucks)):
        location_coordinates=(latitudes[i], longitudes[i])
        distances_from_first_stop.append (distance.distance(first_stop, location_coordinates).miles)
    name_vs_distance_dict={}
    for i in range(len(Food_trucks)):
        name_vs_distance_dict[Food_trucks[i]]=distances_from_first_stop[i]
    return sorted(name_vs_distance_dict.items(), key=operator.itemgetter(1))[1:6]


In [98]:
distances_from_cluster_truck=[]
cluster_truck=(39.7944707585175, -86.1562887527601)

#tour_start_time = 9 am

for i in range (0, len(df_food_trucks)):
    location_coordinates=(latitudes[i], longitudes[i])
    distances_from_cluster_truck.append (distance.distance(cluster_truck, location_coordinates).miles)
    
name_vs_distance_dict={}
for i in range(len(Food_trucks)):
    name_vs_distance_dict[Food_trucks[i]]=distances_from_cluster_truck[i]
top_5_nearest_food_trucks = sorted(name_vs_distance_dict.items(), key=operator.itemgetter(1))[1:6]


In [66]:
top_5_nearest_food_trucks

[('Tortas Guicho Food Truck', 1.4506468656720062),
 ('Spice Box', 1.679239546442854),
 ('General American Donut Company', 2.70411829382776),
 ('Fiesta on the go', 3.1155716312108934),
 ('Felix Tacos Truck', 3.265930354667434)]

### Since Tortas Guicho Food Truck is the closest and is operational until 2 pm, we head there next. We assume that at each stop, we spend 2 hours. 

### Cluster Truck ---> Tortas Guicho Food Truck

### Now, it makes sense to have minimum deviation in the direction when we go from the second food truck to the third
### We ensure that by taking the slope of the line that connect stop1 & stop2 and choose the third one based on min slope difference

In [71]:
#slope of the line joining stop 1 & stop 2
slope_calculator(39.7944707585175, -86.1562887527601, 39.7736299481022, -86.1526741418483)

-0.17343907649373067

In [86]:
#The third stop should follow two conditions:
# 1) The distance from 2 to 3 should be amongst the top 5 minimum possible options
# 2) Food Truck 3 should be open when we get there

In [87]:
Identify_top_5_closest('Tortas Guicho Food Truck', df_food_trucks)

[('Spice Box', 0.2371901647576048),
 ('General American Donut Company', 1.2535877361278533),
 ('Cluster Truck', 1.4506468656720062),
 ('Felix Tacos Truck', 3.1073419702599474),
 ('Fiesta on the go', 4.260588644048227)]

In [81]:
df_food_trucks['Latitude'][df_food_trucks['Name']=='Tortas Guicho Food Truck'].iloc[0]

39.7736299481022

### Next, we head to Spice Box by walk as its less than 0.3 miles and is in operation at 1 pm

### Cluster Truck ---> Tortas Guicho Food Truck ----> Spice Box

In [85]:
Identify_top_5_closest('Spice Box', df_food_trucks)

[('Tortas Guicho Food Truck', 0.2371901647576048),
 ('General American Donut Company', 1.0302196527037202),
 ('Cluster Truck', 1.679239546442854),
 ('Felix Tacos Truck', 3.0762335389574935),
 ('Fiesta on the go', 4.490389918247281)]

### Day_1 : Cluster Truck ---> Tortas Guicho Food Truck ----> Spice Box ---> General American Donut Company

In [91]:
Identify_top_5_closest('Mi Lindo Penjamo', df_food_trucks)

[('Pico de Gallo Food truck', 6.81474622379365),
 ('Fiesta on the go', 8.087264983744033),
 ('Cluster Truck', 9.214010595248467),
 ('Felix Tacos Truck', 10.051475030085294),
 ('Tortas Guicho Food Truck', 10.627465659621294)]

In [95]:
Identify_top_5_closest('Felix Tacos Truck', df_food_trucks)

[('Spice Box', 3.0762335389574935),
 ('Tortas Guicho Food Truck', 3.1073419702599474),
 ('Pico de Gallo Food truck', 3.2463521391082653),
 ('Cluster Truck', 3.265930354667434),
 ('Birrieria Iturbidense', 3.2698464420088604)]

In [101]:
Identify_top_5_closest('Tortas Guicho Food Truck', df_food_trucks)

[('Spice Box', 0.2371901647576048),
 ('General American Donut Company', 1.2535877361278533),
 ('Cluster Truck', 1.4506468656720062),
 ('Felix Tacos Truck', 3.1073419702599474),
 ('Fiesta on the go', 4.260588644048227),
 ('Pico de Gallo Food truck', 4.711527621068245),
 ('Birrieria Iturbidense', 6.213792390798383)]

### On Day 2, we start at 4 pm and proceed to visit food trucks through the evening

In [105]:
Identify_top_5_closest('Pico de Gallo Food truck', df_food_trucks)

[('Felix Tacos Truck', 3.2463521391082653),
 ('Cluster Truck', 3.713879287830699),
 ('Tortas Guicho Food Truck', 4.711527621068245),
 ('Spice Box', 4.8467272040211755),
 ('Fiesta on the go', 5.232858025379438)]

In [106]:
Identify_top_5_closest('Felix Tacos Truck', df_food_trucks)

[('Spice Box', 3.0762335389574935),
 ('Tortas Guicho Food Truck', 3.1073419702599474),
 ('Pico de Gallo Food truck', 3.2463521391082653),
 ('Cluster Truck', 3.265930354667434),
 ('Birrieria Iturbidense', 3.2698464420088604)]

In [107]:
Identify_top_5_closest('Birrieria Iturbidense', df_food_trucks)

[('Felix Tacos Truck', 3.2698464420088604),
 ('Pico de Gallo Food truck', 5.347656288465831),
 ('Spice Box', 6.124816071922045),
 ('Tortas Guicho Food Truck', 6.213792390798383),
 ('General American Donut Company', 6.224308326392008)]

## Day_2: Pico de Gallo ---> Felix Tacos Truck ---> Birrieria Iturbidense